# 6. 학습관련 기술들  

# 6.1 매개변수 갱신  

최적화(optimization): 가능한 한 손실함수가 잔은 매개변수의 최적값을 찾는 것  
매개변수의 공간이 매우 넓고 복잡하여 매우 어려움  
따라서 미분을 이용한 확률적 경사하강법 (SGD)를 많이 사용  
앞으로는 다른 최적화 기법에 대해서도 다룰 것이다.  

## 6.1.1 모험가 이야기  

## 6.1.2 확률적 경사 하강법(SGD)  

SGD를 수식으로 쓰면 다음과 같다.  

<img src = "./image/e_6_1.png">  

η(eta): learning rate / W:  가중치  
위 식을 파이썬으로 구현해 보자.  

In [3]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = LookupError
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -=seelf.lr * grads[key]

SGD 클래스를 사용하면 신경망 매개변수의 진행을 다음과 같이 실행한다.  

```python

network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) # 미니배치
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...
```



매개변수 갱신자: optimizer  
이처럼 촤적화를 담당하는 클래스를 분리해 구현하면 기능을 모듈화하기 좋음

## 6.1.3 SGD의 단점  

SGD는 단순하고 구현이 쉽지만 문제애 따라서 비효울적  
SGD의 단점을 알아보고자 다믕 함수의 최솟값을 구해보자.  

<img src = "./image/e_6_2.png">  

이 함수를 좌표로 보면 다음과 같은 모습을 보인다.  

<img src = "./image/fig_6_1.png" width = "40%">  

x축 방향의 기울기는 완만하지만 y축 기울기의 방향은 가파르다.  
식에서 최솟값이 되는 장소는 (x, y) = (0,0) 이지만, 아래 그림을 보면 기울기는 대부분 (0,0)을 가리키지 않고있다.  

<img src = "./image/fig_6_2.png" width="40%">  

함수에 SGD를 적용해 보자. 담색의 시작 장소(초깃값)는 (x, y) = (-7.0, -2.0)으로 했다.  

<img src = "./image/fig_6_3.png" width="40%">  

최솟값까지 지그재그로 이동하여 비효율적이다.  
즉 SGD의 단점은 비등방성(anisotropy) 함수(방향에 따라 성질, 즉 여기에서는 기울기가 달라지는 함수)에서는 탐색 경로가 비효율적이라는 것이다.  
근본적인 원인은 기울어진 방향이 본래의 최솟값과 다른 방향을 가리켜서라는 점도 생각해 봐야한다.  
단점을 개선해 줄 수 있는 모멘텀, AdaGrad, Adam이라는 세 방법을 살펴보자.  

## 6.1.4 모멘텀

모멘텀(Momentum): 운동량을 뜻하는 단어로 물리와 관계있음, 수식으로는 다음과 같이 쓴다.  

<img src = "./image/e_6_3.png">
<img src = "./image/e_6_4.png">  

v라는 새로운 변수는 물리에서 속도(velocity)에 해당한다.  
모멘텀은 마치 공이 바닥의 기울기에 따라 구르는 듯한 움직임을 보인다.  

<img src="./image/fig_6_4.png">  

αv 항은 물체가 아무런 힘을 받지 않으면 서서히 하강시키는 역할을 한다.  
(α는 0.9등의 값으로 설정한다.) 모멘텀을 파이썬으로 구현해보자.  

In [2]:
import numpy as np 

class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr =lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.seros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

v는 속도이다. v는 초기에 아무 값도 없지만 update() 처음 호출시 매개변수와 같은구조의 데이터를 딕셔너리 변수로 저장한다. 
나머지는 위의 식들을 그대로 코드로 옮겨 놓았다.  
이제 모멘텀을 이용해서 SGD로 풀었던 최적화 문제를 풀어보면 다음과 같다.  

<img src = "./image/fig_6_5.png" width="40%">  

아직 지그재그의 느낌이 있지만 그 폭이 줄어들고 조금더 일정하다.  

## 6.1.5 AdaGrad  

신경망 학습에서 학습률의 값이 중요하다.  